### Imports Defined

In [ ]:
from datetime import datetime
from dateutil import relativedelta
import DBcm
from gazpacho import get,Soup
from ordered_set import OrderedSet
import re

### Website to be scraped

In [ ]:
URL = "https://en.wikipedia.org/wiki/Taoiseach"

In [ ]:
html = get(URL)

In [ ]:
soup = Soup(html)

In [ ]:
tables = soup.find('table')

In [ ]:
for n, t in enumerate(tables):
    if t.attrs['class'] == "wikitable":
        nrows = t.find("tr")
        if isinstance(nrows, list):
            print(f"Table {n} has {len(nrows)} rows.")
        else:
            print(f"Table {n} is of type: {type(nrows)}.")

In [ ]:
data = tables[2]

In [ ]:
rows = data.find('tr')

### Function to convert Soup object into list

In [ ]:
def soup_to_list(text):
    
    new_list = []
    new_list.append(text)
    
    return new_list

### Function to scrape inner URL

In [ ]:
def inner_url_bday_fetch(name):
    
    address = 'https://en.wikipedia.org/'    
    search_str = name.find('a',mode='first')
    split_val = str(search_str).split('"')
    inner_url_val =  ''.join([address,split_val[1]])
    
    html_inner = get(inner_url_val)
    soup_inner = Soup(html_inner)
    
    table_inner = soup_inner.find('table')
    
    for t in table_inner:
        if t.attrs['class'] == "infobox vcard":
            nrows_inner = t.find("tr")
            
            for i in nrows_inner:
                result = i.find('span',{'class':'bday'})
                
                if not isinstance(result,list):
                    bday_value = result.text#.replace('-',' ')
                    
                    bday_value_formatted = datetime.strptime(bday_value, '%Y-%m-%d')
    
    return bday_value_formatted

### Function to get the Constituency name

In [ ]:
def constituency_fetch(name):
    
    a_list = name.find('a')    
    constituency_list = []
    del a_list[0]
    
    for i in a_list :
        val = re.findall(r'(?<=title=")[^(]*',str(i))
        val_in_string = val[0].strip()
        constituency_list.append(val_in_string)
        
    if len(constituency_list) > 1:
        constituency = ', '.join(constituency_list)
        
    else:
        constituency = constituency_list[0]
        
    return constituency

### Function to extract the date vales

In [ ]:
def extract_year(start_day,start_year,end_day,end_year):
    
    search_str = '<br />'
    year_values = []
    if not end_day == 'Incumbent':
        html_row_list = [str(start_year),str(end_year)]
    
        for i in html_row_list:
            start_ind = i.find(search_str) + len(search_str)
            end_ind = start_ind + 4
            year_values.append(i[start_ind:end_ind])
        
        term_start_date = ' '.join([start_day,year_values[0]])
        term_end_date = ' '.join([end_day,year_values[1]])
    
        term_start_date_formatted = datetime.strptime(term_start_date, '%d %B %Y')
        term_start_end_formatted = datetime.strptime(term_end_date, '%d %B %Y')
        
    else:
        html_row_list = str(start_year)
        
        start_ind = html_row_list.find(search_str) + len(search_str)
        end_ind = start_ind + 4
        year_value = html_row_list[start_ind:end_ind]
        
        term_start_date = ' '.join([start_day,year_value])
        
        term_start_date_formatted = datetime.strptime(term_start_date, '%d %B %Y')
        term_start_end_formatted = datetime.today()
        
    return term_start_date_formatted, term_start_end_formatted

### Scraping the data from the URL and storing into a list named the_data

In [ ]:
the_data = []

first_three_rowspan = 1
party_rowspan = 1
term_rowspan = 1
composition_rowspan = 1
vp_rowspan = 1
elect_rowspan = 1 

previous_name = ''
previous_constituency = ''
previous_start = ''
previous_end = ''
previous_party = ''
previous_term = ''
previous_composition = ''
previous_vp = ''
previous_elect = ''
previous_bday = ''
                
for i, row in enumerate(rows[1:]):
    r = row.find('td')
    
    if (isinstance(r,list) and len(r) > 1):
        for j in r:
            if j.text == '':
                r.remove(j)
                
        if first_three_rowspan == 1 and party_rowspan == 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan == 1 and elect_rowspan == 1:
            
            name = r[0].text
            bday_val = inner_url_bday_fetch(r[0])
            
            constituency = constituency_fetch(r[0])
            
            start = r[1].text
            end = r[2].text
            start_date, end_date = extract_year(start,r[1],end,r[2])
            
            if "rowspan" in r[1].attrs:
                first_three_rowspan = int(r[1].attrs["rowspan"])
                previous_name = name
                previous_constituency = constituency
                previous_start = start_date
                previous_end = end_date
                previous_bday = bday_val
            
            party = r[3].text
            if "rowspan" in r[3].attrs:
                party_rowspan = int(r[3].attrs["rowspan"])
                previous_party = party
            
            term = r[4].text
            if "rowspan" in r[4].attrs:
                term_rowspan = int(r[4].attrs["rowspan"])
                previous_term = term

            composition = r[5].text
            if "rowspan" in r[5].attrs:
                composition_rowspan = int(r[5].attrs["rowspan"])
                previous_composition = composition
                
            vp = r[6].text
            if "rowspan" in r[6].attrs:
                vp_rowspan = int(r[6].attrs["rowspan"])
                previous_vp = vp
                
            elect = r[7].text
            if "rowspan" in r[7].attrs:
                elect_rowspan = int(r[7].attrs["rowspan"])
                previous_elect = elect

        elif first_three_rowspan > 1 and party_rowspan == 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan > 1 and elect_rowspan == 1:
            
            first_three_rowspan -= 1
            vp_rowspan -= 1
            
            party = r[0].text
            if "rowspan" in r[0].attrs:
                party_rowspan = int(r[0].attrs["rowspan"])
                previous_party = party
            
            term = r[1].text
            if "rowspan" in r[1].attrs:
                term_rowspan = int(r[1].attrs["rowspan"])
                previous_term = term
            
            composition = r[2].text
            if "rowspan" in r[2].attrs:
                composition_rowspan = int(r[2].attrs["rowspan"])
                previous_composition = composition
            
            elect = r[3].text
            if "rowspan" in r[3].attrs:
                elect_rowspan = int(r[3].attrs["rowspan"])
                previous_elect = elect
                
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan > 1 and vp_rowspan == 1 and elect_rowspan == 1:
                        
            first_three_rowspan -= 1
            party_rowspan -= 1
            composition_rowspan -= 1
            
            term = r[0].text
            if "rowspan" in r[0].attrs:
                term_rowspan = int(r[0].attrs["rowspan"])
                previous_term = term
                
            vp = r[1].text
            if "rowspan" in r[1].attrs:
                vp_rowspan = int(r[1].attrs["rowspan"])
                previous_vp = vp
                
            elect = r[2].text
            if "rowspan" in r[2].attrs:
                elect_rowspan = int(r[2].attrs["rowspan"])
                previous_elect = elect
        
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan > 1 and vp_rowspan > 1 and elect_rowspan == 1:
            
            first_three_rowspan -= 1
            party_rowspan -= 1
            composition_rowspan -= 1
            vp_rowspan -= 1
            
            term = r[0].text
            if "rowspan" in r[0].attrs:
                term_rowspan = int(r[0].attrs["rowspan"])
                previous_term = term
            
            elect = r[1].text
            if "rowspan" in r[1].attrs:
                elect_rowspan = int(r[1].attrs["rowspan"])
                previous_elect = elect
                  
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan > 1 and vp_rowspan > 1 and elect_rowspan > 1:
            
            first_three_rowspan -= 1
            party_rowspan -= 1
            composition_rowspan -= 1
            vp_rowspan -= 1
            elect_rowspan -= 1
            
            term = r[0].text
            if "rowspan" in r[0].attrs:
                term_rowspan = int(r[0].attrs["rowspan"])
                previous_term = term
        
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan > 1 and elect_rowspan == 1:
            
            first_three_rowspan -= 1
            party_rowspan -= 1
            vp_rowspan -= 1
            
            term = r[0].text
            if "rowspan" in r[0].attrs:
                term_rowspan = int(r[0].attrs["rowspan"])
                previous_term = term
                
            composition = r[1].text
            if "rowspan" in r[1].attrs:
                composition_rowspan = int(r[1].attrs["rowspan"])
                previous_composition = composition
                
            elect = r[2].text
            if "rowspan" in r[2].attrs:
                elect_rowspan = int(r[2].attrs["rowspan"])
                previous_elect = elect
            
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan == 1 and elect_rowspan == 1:
            
            first_three_rowspan -= 1
            party_rowspan -= 1
                      
            term = r[0].text
            if "rowspan" in r[0].attrs:
                term_rowspan = int(r[0].attrs["rowspan"])
                previous_term = term
                
            composition = r[1].text
            if "rowspan" in r[1].attrs:
                composition_rowspan = int(r[1].attrs["rowspan"])
                previous_composition = composition
                
            vp = r[2].text
            if "rowspan" in r[2].attrs:
                vp_rowspan = int(r[2].attrs["rowspan"])
                previous_vp = vp
                
            elect = r[3].text
            if "rowspan" in r[3].attrs:
                elect_rowspan = int(r[3].attrs["rowspan"])
                previous_elect = elect
            
        elif first_three_rowspan == 1 and party_rowspan == 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan == 1 and elect_rowspan > 1:
            
            elect_rowspan -= 1
            
            name = r[0].text
            bday_val = inner_url_bday_fetch(r[0])
            
            constituency = constituency_fetch(r[0])
            
            start = r[1].text
            end = r[2].text       
            start_date, end_date = extract_year(start,r[1],end,r[2])  
            
            if "rowspan" in r[1].attrs:
                first_three_rowspan = int(r[1].attrs["rowspan"])
                previous_name = name
                previous_constituency = constituency
                previous_start = start_date
                previous_end = end_date
                previous_bday = bday_val
            
            party = r[3].text
            if "rowspan" in r[3].attrs:
                party_rowspan = int(r[3].attrs["rowspan"])
                previous_party = party
            
            term = r[4].text
            if "rowspan" in r[4].attrs:
                term_rowspan = int(r[4].attrs["rowspan"])
                previous_term = term

            composition = r[5].text
            if "rowspan" in r[5].attrs:
                composition_rowspan = int(r[5].attrs["rowspan"])
                previous_composition = composition
                
            vp = r[6].text
            if "rowspan" in r[6].attrs:
                vp_rowspan = int(r[6].attrs["rowspan"])
                previous_vp = vp
                   
        elif first_three_rowspan == 1 and party_rowspan == 1 and term_rowspan == 1 and composition_rowspan == 1 and vp_rowspan > 1 and elect_rowspan > 1:
            
            vp_rowspan -= 1
            elect_rowspan -= 1
            
            name = r[0].text
            bday_val = inner_url_bday_fetch(r[0])
            
            constituency = constituency_fetch(r[0])
            
            start = r[1].text
            end = r[2].text
            
            start_date, end_date = extract_year(start,r[1],end,r[2])
            
            if "rowspan" in r[1].attrs:
                first_three_rowspan = int(r[1].attrs["rowspan"])
                previous_name = name
                previous_constituency = constituency
                previous_start = start_date
                previous_end = end_date
                previous_bday = bday_val
            
            party = r[3].text
            if "rowspan" in r[3].attrs:
                party_rowspan = int(r[3].attrs["rowspan"])
                previous_party = party
            
            term = r[4].text
            if "rowspan" in r[4].attrs:
                term_rowspan = int(r[4].attrs["rowspan"])
                previous_term = term

            composition = r[5].text
            if "rowspan" in r[5].attrs:
                composition_rowspan = int(r[5].attrs["rowspan"])
                previous_composition = composition
            
        elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan > 1 and composition_rowspan > 1 and vp_rowspan == 1 and elect_rowspan > 1:
            
            first_three_rowspan -= 1
            party_rowspan -= 1
            term_rowspan -= 1
            composition_rowspan -= 1
            elect_rowspan -= 1
            
            vp = r[0].text
            if "rowspan" in r[0].attrs:
                vp_rowspan = int(r[0].attrs["rowspan"])
                previous_vp = vp
        
        the_data.append(
                    (
                        name,
                        constituency,
                        start_date,
                        end_date,
                        party,
                        term,
                        composition,
                        vp,
                        int(elect),
                        bday_val
                    )
            )
    
    elif (not isinstance(r,list)):

        value = r.text
        if len(value) > 0:
            r = soup_to_list(value)
            
            if first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan == 1 and composition_rowspan > 1 and vp_rowspan > 1 and elect_rowspan > 1:
                     
                first_three_rowspan -= 1
                party_rowspan -= 1
                composition_rowspan -= 1
                vp_rowspan -= 1
                elect_rowspan -= 1
            
                term = r[0]
                                
            elif first_three_rowspan > 1 and party_rowspan > 1 and term_rowspan > 1 and composition_rowspan > 1 and vp_rowspan == 1 and elect_rowspan > 1:
            
                first_three_rowspan -= 1
                party_rowspan -= 1
                term_rowspan -= 1
                composition_rowspan -= 1
                elect_rowspan -= 1
            
                vp = r[0]
       
            the_data.append(
                    (
                        name,
                        constituency,
                        start_date,
                        end_date,
                        party,
                        term,
                        composition,
                        vp,
                        int(elect),
                        bday_val
                    )
            )

In [ ]:
del the_data[36]
del the_data[-2]

### Database configuration

In [ ]:
config = {
        'user' : 'taoiuser',
        'password': 'taoipasswd',
        'host' : '127.0.0.1',
        'database' : 'taoiseachDB'
}

### Inserting values into database table

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """INSERT INTO taoiseach (Name,Constituency,Start_Date,End_Date,Party,Term,Council,Vice_President,Elected,DOB) 
             VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cursor.executemany(SQL,the_data)

### Provide a list of constituencies in Ireland which have produced a Taoiseach

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT DISTINCT(Constituency) FROM taoiseach """
    cursor.execute(SQL)
    distinct_constituencies = cursor.fetchall()
    
print (distinct_constituencies)

### Which political party has produced the most individual Taoiseach?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT Party FROM taoiseach GROUP BY(Party) ORDER BY COUNT(*) DESC LIMIT 1 """
    cursor.execute(SQL)
    party_name = cursor.fetchall()
    
print (party_name)

### Provide the list of names for those politicians who previously held the position of “Vice President” or “Tánaiste” prior to becoming Taoiseach.

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT DISTINCT(Name) FROM taoiseach t1 WHERE EXISTS (SELECT 1
                                                                    FROM taoiseach t2
                                                                    WHERE t2.Vice_President = t1.Name) """
    cursor.execute(SQL)
    Taoiseach_and_VP = cursor.fetchall()

print (Taoiseach_and_VP)

### Which political party held the office of Taoiseach for the longest amount of overall time, and for how long (in years, months, and days)?

In [ ]:
def longest_overall_term(the_data):
    
    period = {}
    distinct_party = {a[5] for a in the_data}
    
    for i in distinct_party:
        year_sum = 0
        month_sum = 0
        day_sum = 0
        for a in the_data:
            if i == a[5]:
                diff = relativedelta.relativedelta(a[4], a[3])
            
                year_sum += diff.years
                month_sum += diff.months
                day_sum += diff.days
            
        period[i] = [year_sum,month_sum,day_sum] 

    longest_term = max(period.values())
        
    for k,v in period.items():
        if v == longest_term:
            party_name = k

    value = f"The longest overall term is of {longest_term[0]} years, {longest_term[1]} months, {longest_term[2]} days held by {party_name}."
    
    return value

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT * FROM taoiseach """
    cursor.execute(SQL)
    all_data = cursor.fetchall()
    
longest_overall_term(all_data)

### Which individual politician held the office of Taoiseach for the longest amount of uninterrupted time, and for how long (in years, months, and days)?

### Which individual politician held the office of Taoiseach for the shortest amount of time, and for how long (in years, months, and days)?

In [ ]:
def term_of_politician(the_data):
    
    uninterrupted_period = {}
    ordered_set = OrderedSet()    

    for i in the_data:
        a = (i[1],i[2],i[3],i[4])
        ordered_set.add(a)

    for n,i in enumerate(list(ordered_set)):
    
        current_val = i[0]    
        if (ordered_set[n-1][0] == current_val):
         
            diff = relativedelta.relativedelta(ordered_set[n][3], ordered_set[n][2])
        
            year_sum += diff.years
            month_sum += diff.months
            day_sum += diff.days
            
            uninterrupted_period[current_val + str(n)] = [year_sum,month_sum,day_sum] 
            
        else:
        
            year_sum = 0
            month_sum = 0
            day_sum = 0
            
            diff = relativedelta.relativedelta(ordered_set[n][3], ordered_set[n][2])
            year_sum += diff.years
            month_sum += diff.months
            day_sum += diff.days
            
            uninterrupted_period[current_val + str(n)] = [year_sum,month_sum,day_sum]
            
        longest_term = max(uninterrupted_period.values())
        shortest_term = min(uninterrupted_period.values())
        
        for k,v in uninterrupted_period.items():
            if v == longest_term:
                plolitician_longest_term = re.findall(r'[\D]+',k)[0]
                
            elif v == shortest_term:
                plolitician_shortest_term = re.findall(r'[\D]+',k)[0]
                
    value1 = f"The longest uninterrupted term is of {longest_term[0]} years, {longest_term[1]} months, {longest_term[2]} days was held by {plolitician_longest_term}."
    value2 = f"The shortest term is {shortest_term[0]} years, {shortest_term[1]} months, {shortest_term[2]} days was held by {plolitician_shortest_term}."
    
    return value1,value2

In [ ]:
term_of_politician(all_data)

### Which political party has held the office of Taoiseach for the largest number of Dáils?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT Party FROM taoiseach ORDER BY Elected DESC LIMIT 1 """
    cursor.execute(SQL)
    largest_dail_party = cursor.fetchall()
    
print(largest_dail_party)

### Which individual politician has held the office of Taoiseach for the most number of Dáils?

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = """ SELECT Name FROM taoiseach ORDER BY Elected DESC LIMIT 1 """
    cursor.execute(SQL)
    largest_dail_politician = cursor.fetchall()
    
print(largest_dail_politician)

### What age was each Taoiseach on the day they assumed office? 

In [ ]:
data = OrderedSet( [( i[1] ,relativedelta.relativedelta(i[3],i[-1]).years ) for i in all_data] )

for i in data:
    print (f"{i[0]} was {i[1]} on the day he assumed office")

### On the last day of their term of office, which politician was the oldest?

In [ ]:
def max_ageed_politician_on_last_day(the_data):
    
    politician_age_on_last_day = {}
    data = OrderedSet( [( i[1] ,relativedelta.relativedelta(i[4],i[-1]).years ) for i in the_data] )
   
    for n,i in enumerate(data):
        politician_age_on_last_day[i[0]+str(n)] = i[1]
        
    max_age = max(politician_age_on_last_day.values())
    
    for k,v in politician_age_on_last_day.items():
        if v == max_age:
            name = re.findall(r'[\D]+',k)[0]
            
    value = f"The oldest aged ploitician on the last day of his term was {name} aged at {max_age} years"
    
    return value
    

In [ ]:
max_ageed_politician_on_last_day(all_data)

### In your view, and based on statistics calculated from your scraped data, which Irish political party is the most successful?  Show and describe your calculations

In my view the most successful party has to be the one which has been in the office of Taoiseach for the longest duration 
of time. This is calculated from the function <b>longest_overall_term</b>, defined above.
   